In [4]:
import sys, os

# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

from utils.experiment_tracker import ExperimentTracker
from utils.constants import METRICS_DB_PATH, CLEANED_DIR
from utils.model_evaluator import ModelEvaluator
from utils.model_table import ModelComparativelTable  


mcp = ModelComparativelTable()
model_pre_study_path = "../../data/ml_pre_study_metrics/model_metrics.csv"
mcp.display_model_summary_pre_study(model_pre_study_path)

mcp.display_model_summary_pre_study
mcp.display_model_summary()


=== Model Evaluation Summary ===


,model,mae,rmse,r2,best,type,rank_r2,rmse/mae
0,Linear Regression (All Features),85 510.11 €,121 149.58 €,0.636600,,Linear,29,1.420000
1,Random Forest (All Features),65 705.67 €,98 940.13 €,0.757600,,Tree,24,1.510000
2,Linear Regression (Top Features),96 534.10 €,136 104.16 €,0.541300,,Linear,30,1.410000
3,Random Forest (Top Features),67 309.43 €,101 408.12 €,0.745400,,Tree,26,1.510000
4,XGBoost (All Features),66 302.09 €,97 431.35 €,0.764900,,Boosting,22,1.470000
5,XGBoost (Top RF Features),65 661.18 €,97 742.14 €,0.763400,,Boosting,23,1.490000
6,GradientBoosting (All Features),76 999.33 €,110 878.68 €,0.695600,,Other,28,1.440000
7,GradientBoosting (Top RF Features),76 517.08 €,110 878.68 €,0.696300,,Other,27,1.450000
8,LightGBM (All Features),65 548.48 €,97 417.23 €,0.765000,,Boosting,21,1.490000
9,LightGBM (Top RF Features),67 205.03 €,99 709.18 €,0.753800,,Boosting,25,1.480000



👉 Best model based on 'r2': CatBoost + Optuna CV (Top RF Features) – optimized ✓
=== Model Evaluation Summary ===


,id,timestamp,model,dataset,experiment,cleaning_version_id,mae,rmse,r2,best,type,rank_r2,rmse/mae
0,38,2025-06-29 01:40:22,Linear Regression (All Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,Linear Regression (All Features),202506290139,76 898.76 €,109 935.46 €,0.723000,,Linear,7,1.430000
1,39,2025-06-29 01:40:47,Random Forest (All Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,Random Forest (All Features),202506290139,68 324.02 €,101 428.25 €,0.764200,,Tree,5,1.480000
2,40,2025-06-29 01:40:55,Random Forest (Top 10 Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,Random Forest (Top 10 Features),202506290139,71 734.61 €,106 140.03 €,0.741800,,Tree,6,1.480000
3,41,2025-06-29 01:56:44,XGBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,XGBoost with Optuna (All Features),202506290139,29 964.75 €,42 415.21 €,0.958800,✓,Boosting,1,1.420000
4,42,2025-06-29 01:56:44,XGBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,XGBoost with Optuna (Top RF Features),202506290139,31 236.38 €,44 189.81 €,0.955300,,Boosting,3,1.410000
5,43,2025-06-29 02:33:46,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,CatBoost with Optuna (All Features),202506290139,31 541.17 €,43 695.99 €,0.956300,,Boosting,2,1.390000
6,44,2025-06-29 02:33:46,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_0139.csv,CatBoost with Optuna (Top RF Features),202506290139,43 624.67 €,60 783.29 €,0.915400,,Boosting,4,1.390000



👉 Best model based on R²: XGBoost + Optuna CV (All Features) ✓
